In [ ]:
# Prerparing dataset triplet for siamese network
import numpy as np
from tensorflow.keras.datasets import mnist

(x_train, y_train), (_, _) = mnist.load_data()

num_classes = 10
approx_samples_per_class = 15

reshaped_x_train = []
reshaped_y_train = []
anchor_idx = []
positive_idx = []
negative_idx = []
anchor_y = []
positive_y = []
negative_y = []

for i in range(num_classes):
    class_indices = np.where(y_train == i)[0]
    num_samples = min(approx_samples_per_class, class_indices.shape[0])
    selected_indices = np.random.choice(class_indices, size=num_samples, replace=False)
    reshaped_x_train.append(x_train[selected_indices])
    reshaped_y_train.append(y_train[selected_indices])

reshaped_x_train = np.array(reshaped_x_train)
reshaped_y_train = np.array(reshaped_y_train)
x_train = reshaped_x_train
print("Reshaped x_train shape:", reshaped_x_train.shape)
print("Reshaped y_train shape:", reshaped_y_train.shape)
for i in range(len(x_train)):
    for j in range(len(x_train[i])):
        for k in range(j+1, len(x_train[i])):
            for l in range(len(x_train)):
                if l==i:
                    continue
                for m in range(len(x_train[l])):
                    anchor_idx.append(x_train[i,j])
                    positive_idx.append(x_train[i,k])
                    negative_idx.append(x_train[l,m])
                    anchor_y.append(i)
                    positive_y.append(i)
                    negative_y.append(l)


print(len(anchor_idx))
anchor_idx=np.array(anchor_idx)
positive_idx=np.array(positive_idx)
negative_idx=np.array(negative_idx)
anchor_y=np.array(anchor_y)
positive_y=np.array(anchor_y)
negative_y=np.array(negative_y)


In [ ]:
# Import stuffs
import tensorflow as tf
from tensorflow import keras
import keras
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Input, Flatten, Dense, Lambda, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import Sequential


In [ ]:
# Create model
def VGG16():
    model = tf.keras.Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28,28,1)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='sigmoid'))

    return model

feature = VGG16()
image_a = Input(shape=(28,28), name='image_input_a')
image_b = Input(shape=(28,28), name='image_input_b')
image_c = Input(shape=(28,28), name='image_input_c')

feature_vector_A = feature(image_a)
feature_vector_B = feature(image_b)
feature_vector_C = feature(image_c)

def euclidean_distance(a, b):
    return tf.sqrt(tf.reduce_sum(tf.square(a - b), axis=-1))

class CustomLayer(keras.layers.Layer):
    def custom_loss(self, output_intermediate_a, output_intermediate_b, output_intermediate_c):
        output_a = output_intermediate_a
        output_b = output_intermediate_b
        output_c = output_intermediate_c
        distance_positive = tf.reduce_sum(tf.square(output_a[:,0:1000] - output_b[:,0:1000]), axis=1)
        distance_negative = tf.reduce_sum(tf.square(output_a[:,0:1000] - output_c[:,0:1000]), axis=1)
        loss = tf.maximum(distance_positive - distance_negative + 0.1, 0.0)
        loss = tf.reduce_mean(loss)
        return K.mean(loss)

    def call(self, inputs):
        output_intermediate_a = inputs[0]
        output_intermediate_b = inputs[1]
        output_intermediate_c = inputs[2]
        loss = self.custom_loss(output_intermediate_a, output_intermediate_b, output_intermediate_c)
        self.add_loss(loss, inputs=inputs)
        return inputs
y = CustomLayer()([feature_vector_A, feature_vector_B, feature_vector_C])

discriminator = Sequential([
    Dense(10, input_shape=(4096,), activation='sigmoid')
])

class_vector_A = discriminator(y[0])
class_vector_B = discriminator(y[1])
class_vector_C = discriminator(y[2])


model2 = keras.Model([image_a, image_b, image_c], [class_vector_A, class_vector_B, class_vector_C])


def custom_loss(y_true, y_pred):

    y_true_onehot_1 = tf.one_hot(tf.cast(y_true[:,0], dtype=tf.int32), depth=10)
    y_true_onehot_1 = tf.squeeze(y_true_onehot_1, axis=0)
    y_pred_onehot_1 = y_pred[:,0]
    loss1 = tf.keras.losses.CategoricalCrossentropy()(y_true_onehot_1, y_pred_onehot_1)
    y_true_onehot_2 = tf.one_hot(tf.cast(y_true[:,1], dtype=tf.int32), depth=10)
    y_true_onehot_2 = tf.squeeze(y_true_onehot_2, axis=0)
    y_pred_onehot_2 = y_pred[:,1]
    loss2 = tf.keras.losses.CategoricalCrossentropy()(y_true_onehot_2, y_pred_onehot_2)
    y_true_onehot_3 = tf.one_hot(tf.cast(y_true[:,2], dtype=tf.int32), depth=10)
    y_true_onehot_3 = tf.squeeze(y_true_onehot_3, axis=0)
    y_pred_onehot_3 = y_pred[:,2]
    loss3 = tf.keras.losses.CategoricalCrossentropy()(y_true_onehot_3, y_pred_onehot_3)

    return loss1+loss2+loss3

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()



In [ ]:
model2 = keras.Model([image_a, image_b, image_c], [class_vector_A, class_vector_B, class_vector_C])


In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(0.00005), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.fit([anchor_idx,positive_idx,negative_idx], [to_categorical(anchor_y),to_categorical(positive_y),to_categorical(negative_y)], validation_split=0.2,epochs = 100, batch_size = 2048)

In [ ]:
image = Input(shape=(28,28), name='image_input')
feature_vector = feature(image)
class_vector = discriminator(feature_vector)
model = keras.Model(image, class_vector)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.utils import to_categorical

In [ ]:
model.fit(anchor_idx, to_categorical(anchor_y), epochs = 100, batch_size = 2048,shuffle=True,validation_data=(x_test, to_categorical(y_test)))


In [ ]:
filename = "/content/drive/MyDrive/mnist/Siamesemodel_triple.joblib"
import joblib
joblib.dump(model2, filename)

In [ ]:
import joblib
import tensorflow as tf
from tensorflow.keras.utils import custom_object_scope

with custom_object_scope({'CustomLayer': CustomLayer}):
    loaded_model = joblib.load('/content/drive/MyDrive/mnist/Siamesemodel_triple.joblib')
model = loaded_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape